## Importing necessary libraries

In [12]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN, KMeans
from scipy.spatial import ConvexHull
from scipy.stats import norm
import pickle
import os


## Helper Functions for Data Collection

In [13]:
def collect_data(filename):
    # Open the file and extract locs
    with h5py.File(filename, 'r') as f:
        locs = f['locs']
        x_values = locs['x'][:]
        y_values = locs['y'][:]
    
    data = {'x': x_values, 'y': y_values}
    return pd.DataFrame(data)

def collect_group_data(hdf5_file, dataset_name):
    group_data_list = []
    
    with h5py.File(hdf5_file, 'r') as f:
        dataset = f[dataset_name]
        group_data = dataset['group'][:]
        x_data = dataset['x'][:]
        y_data = dataset['y'][:]
        
        unique_groups = np.unique(group_data)
        
        for group in unique_groups:
            indices = np.where(group_data == group)
            group_dict = {
                'group': int(group),
                'x': x_data[indices].tolist(),
                'y': y_data[indices].tolist()
            }
            group_data_list.append(group_dict)
    
    return group_data_list


## **Clustering Functions** 
- **We perform DBSCAN filtering and K-Means clustering to find clusters within the data. These functions handle the clustering processes and the calculation of the center of mass for each cluster.**

In [14]:
def dbscan_filter(data, eps=0.05, min_samples=5):
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    core_samples = clustering.core_sample_indices_
    return data.iloc[core_samples]

def find_clusters_k_means(data, k):
    kmeans = KMeans(init="k-means++", n_clusters=k, tol=1e-8, n_init=8, max_iter=1000)
    kmeans.fit(data)
    return kmeans.labels_

def find_com(data, labels):
    com = []
    for i in range(max(labels)+1):
        x = np.mean(data['x'][labels == i])
        y = np.mean(data['y'][labels == i])
        com.append((x, y))
    return com


## Geometric Calculations
- **This section contains functions to calculate the minimum bounding rectangle for the clusters and other geometric operations like rotating the clusters and calculating distances.**


In [15]:
def minimum_bounding_rectangle(points):
    pi2 = np.pi / 2
    hull_points = points[ConvexHull(points).vertices]
    edges = hull_points[1:] - hull_points[:-1]
    angles = np.arctan2(edges[:, 1], edges[:, 0])
    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)
    
    rotations = np.vstack([
        np.cos(angles), np.cos(angles - pi2),
        np.cos(angles + pi2), np.cos(angles)
    ]).T.reshape((-1, 2, 2))

    rot_points = np.dot(rotations, hull_points.T)
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    x1, x2 = max_x[best_idx], min_x[best_idx]
    y1, y2 = max_y[best_idx], min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)
    return rval


## Distance Calculations
- **These helper functions help calculate distances between points and lines, which are useful in determining the closest cluster center of mass.**


In [16]:
def distance_to_line(point, line_start, line_end):
    if np.all(line_start == line_end):
        return np.linalg.norm(point - line_start)
    return np.abs(np.cross(line_end - line_start, point - line_start) / np.linalg.norm(line_end - line_start))


## Plotting Helper Function
- **We use this function to visualize different stages of the analysis, including initial data, clusters, centers of mass, and bounding rectangles.**

In [17]:
def plot_helper(data=None, labels=None, com=None, rotated_data=None, rotated_com=None, rotated_rect=None, title="Plot"):
    plt.figure(figsize=(10, 10))
    ax = plt.gca()
    ax.set_facecolor('black')
    ax.invert_yaxis()

    if data is not None and labels is not None:
        plt.scatter(data['x'], data['y'], c=labels, cmap='viridis', s=10, alpha=0.5)
    elif data is not None:
        plt.scatter(data['x'], data['y'], s=10, alpha=0.5)

    if com is not None and len(com) > 0:
        com = np.array(com)
        plt.scatter(com[:, 0], com[:, 1], c='red', s=100, label='COMs')

    if rotated_data is not None and rotated_data.size > 0:
        plt.scatter(rotated_data[:, 0], rotated_data[:, 1], color='cyan', s=10, alpha=0.5)

    if rotated_rect is not None and rotated_rect.size > 0:
        for i in range(4):
            plt.plot([rotated_rect[i, 0], rotated_rect[(i + 1) % 4, 0]], 
                     [rotated_rect[i, 1], rotated_rect[(i + 1) % 4, 1]], 'b-')

    plt.title(title)
    plt.grid(False)
    plt.legend()
    plt.show()


## Process Origami Ratios
- **This function brings everything together by processing the data, clustering it, finding centers of mass, and calculating ratios for further analysis.**


In [18]:
def process_origami_ratio(filename, k):
    data = collect_data(filename)
    plot_helper(data=data, title="Initial Data")

    labels = find_clusters_k_means(data, k)
    plot_helper(data=data, labels=labels, title="K-Means Clustering")

    com = find_com(data, labels)
    plot_helper(data=data, labels=labels, com=com, title="Centers of Mass")

    min_bounding_rect = minimum_bounding_rectangle(np.array(com))
    plot_helper(data=data, labels=labels, com=com, rotated_rect=min_bounding_rect, title="Minimum Bounding Rectangle")


## Save and Load Categories
- **This cell provides functionality to dynamically categorize ratios, save those categories to a file, and load them for future analysis, saving time on re-analysis.**


In [19]:
def input_categories():
    num_categories = int(input("Enter the number of categories: "))
    categories = {}
    for i in range(num_categories):
        category_name = input(f"Enter the name of category {i + 1}: ").strip()
        categories[category_name] = []
    return categories

def categorize_dynamically(group_num, ratio, categories):
    category = input(f"Select a category for Group {group_num} from {list(categories.keys())}: ").strip()
    if category in categories:
        categories[category].append({"group": group_num, "ratio": ratio})
    else:
        print(f"Category {category} not found.")
    return categories

def save_categories(categories, save_file="categories.pkl"):
    with open(save_file, 'wb') as f:
        pickle.dump(categories, f)
    print(f"Categories saved to {save_file}")

def load_categories(save_file="categories.pkl"):
    if os.path.exists(save_file):
        with open(save_file, 'rb') as f:
            categories = pickle.load(f)
        return categories
    else:
        return None


## Gaussian Curve Plotting
- **This function generates Gaussian curves based on the categorized data, which can be used for further analysis and visualization of the data distribution.**


In [20]:
def gaussian_curve_generator(categories):
    gaussian_curves = {}
    for category, groups in categories.items():
        ratios = [group_data["ratio"] for group_data in groups]
        mean = np.mean(ratios)
        std = np.std(ratios)
        x_values = np.linspace(mean - 4*std, mean + 4*std, 1000)
        y_values = norm.pdf(x_values, mean, std)
        gaussian_curves[category] = {"mean": mean, "std": std, "x_values": x_values, "y_values": y_values}
    return gaussian_curves

def plot_gaussian_curve(gaussian_curves, category):
    if category in gaussian_curves:
        curve = gaussian_curves[category]
        plt.plot(curve["x_values"], curve["y_values"], label=f'{category} (mean={curve["mean"]:.3f}, std={curve["std"]:.3f})')
        plt.title(f"Gaussian Curve for {category}")
        plt.xlabel("Ratio")
        plt.ylabel("Probability Density")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print(f"Category '{category}' not found.")
        
def combine_gaussian_curves(gaussian_curve_1, gaussian_curve_2, label1="Curve 1", label2="Curve 2"):
    """
    Combine two Gaussian curves into one plot.
    
    :param gaussian_curve_1: Dictionary with x_values and y_values for the first Gaussian curve.
    :param gaussian_curve_2: Dictionary with x_values and y_values for the second Gaussian curve.
    :param label1: Label for the first curve in the plot.
    :param label2: Label for the second curve in the plot.
    """
    # Plot both Gaussian curves
    plt.plot(gaussian_curve_1["x_values"], gaussian_curve_1["y_values"], label=label1, color='blue')
    plt.plot(gaussian_curve_2["x_values"], gaussian_curve_2["y_values"], label=label2, color='red')
    
    plt.title("Combined Gaussian Curves")
    plt.xlabel("Ratio")
    plt.ylabel("Probability Density")
    plt.legend()
    plt.grid(True)
    plt.show()



# Example of loading saved categories and plotting the Gaussian curve
categories = load_categories("categories.pkl")  # Load previously saved categories
if categories:
    gaussian_curves = gaussian_curve_generator(categories)  # Generate Gaussian curves
    plot_gaussian_curve(gaussian_curves, "Category 1")  # Plot the Gaussian curve for "Category 1"
else:
    print("No saved categories found.")


## Helper functions for processing multiple orgami

In [21]:
def generate_ratio_histogram(ratios, bins=10, title="Histogram of Ratios", xlabel="Ratio", ylabel="Frequency"):
    """
    Generates a histogram of the provided ratios.
    
    :param ratios: List of ratio values to plot.
    :param bins: Number of bins for the histogram.
    :param title: Title of the histogram plot.
    :param xlabel: Label for the x-axis.
    :param ylabel: Label for the y-axis.
    """
    plt.figure(figsize=(10, 6))
    plt.style.use('seaborn-v0_8-darkgrid')
    plt.hist(ratios, bins=bins, color='blue', alpha=0.7, edgecolor='black')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()
    

def plot_ratio_points_and_lines(ratios, title="Points and Lines Plot of Ratios", xlabel="Index", ylabel="Ratio"):
    """
    Plots points for each ratio and connects them with lines to visualize peaks.
    
    :param ratios: List of ratio values to plot.
    :param title: Title of the plot.
    :param xlabel: Label for the x-axis.
    :param ylabel: Label for the y-axis.
    """
    plt.figure(figsize=(20, 12))
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Plotting the points
    plt.plot(range(len(ratios)), ratios, 'o-', color='red', markersize=8, linewidth=2)
    
    # Annotate the peak points
    for i, ratio in enumerate(ratios):
        plt.annotate(f"{ratio:.6f}", (i, ratio), textcoords="offset points", xytext=(0,5), ha='center')
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

def input_categories():
    """
    Predefine category names before processing groups.
    
    :return: A dictionary with empty lists for each category.
    """
    num_categories = int(input("Enter the number of categories: "))
    categories = {}

    for i in range(num_categories):
        category_name = input(f"Enter the name of category {i + 1}: ").strip()
        categories[category_name] = []
    
    return categories

def categorize_dynamically(group_num, ratio, categories):
    """
    Dynamically categorize each group's ratio or stop the process if 'exit' is entered.
    
    :param group_num: The number of the current group being processed.
    :param ratio: The calculated ratio for the group.
    :param categories: The predefined dictionary of categories.
    :return: Updated categories with the group ratio assigned, or a flag indicating an exit.
    """
    category = input(f"Select a category for Group {group_num} from the following: {list(categories.keys())} (type 'exit' to stop): ").strip()

    if category.lower() == 'exit':
        print("Exiting the categorization process...")
        return 'exit'  # Flag to stop further processing

    if category in categories:
        categories[category].append({"group": group_num, "ratio": ratio})
    else:
        print(f"Category {category} does not exist. Please try again.")
        return categorize_dynamically(group_num, ratio, categories)
    
    return categories


## Final prcess function

In [22]:
def process_multiple_origami_ratio_with_categorize_and_gaussian(filename, k, flipped=False, eps=0.25, min_samples=2, save_file="categories.pkl"):
    """
    Extended version that saves the picked categories and Gaussian curves.
    """
    # Check if a saved categories file exists
    if os.path.exists(save_file):
        print(f"Loading saved categories from {save_file}...")
        with open(save_file, 'rb') as f:
            categories = pickle.load(f)
    else:
        # Initialize categories by taking input from the user if no saved file is found
        categories = input_categories()

    # Collect data for each group
    group_data_list = collect_group_data(filename, 'locs')

    ratios = []  # List to store the ratios for each group
    
    # Process each group
    for i, group_data in enumerate(group_data_list, start=1):
        data = pd.DataFrame({
            'x': group_data['x'],
            'y': group_data['y']
        })
        
        # Perform clustering and find centers of mass
        data = dbscan_filter(data, eps=0.25, min_samples=2)
        labels = find_clusters_k_means(data, k)
        com = find_com(data, labels)

        # Find the minimum bounding rectangle for the centers of mass
        min_bounding_rect = minimum_bounding_rectangle(np.array(com))
        
        # Identify the closest side of the bounding rectangle
        closest_side = find_closest_side(np.array(com), min_bounding_rect)
        
        # Determine the rotation angle to align the rectangle
        rotation_angle = find_rotation_angle(min_bounding_rect, closest_side)
        
        # Rotate the data points and centers of mass around the closest side
        rotated_data = rotate_points(data.values, rotation_angle, min_bounding_rect[closest_side])
        rotated_com = rotate_points(np.array(com), rotation_angle, min_bounding_rect[closest_side])
        rotated_rect = rotate_points(min_bounding_rect, rotation_angle, min_bounding_rect[closest_side])
        
        # Adjust the final orientation if needed
        rotated_com, rotated_rect, final_angle = adjust_final_orientation(rotated_com, rotated_rect, rotation_angle, closest_side)
        
        if rotation_angle != final_angle:
            rotated_data = rotate_points(rotated_data, np.pi, find_center_of_rectangle(rotated_rect))

        if flipped:
            rotated_data = np.array([[-x, y] for x, y in rotated_data])
            rotated_com = np.array([[-x, y] for x, y in rotated_com])
            rotated_rect = np.array([[-x, y] for x, y in rotated_rect])

        second_highest_com = find_middle_left_most_com(rotated_com)

        if second_highest_com is None:
            continue

        right_most_com, second_right_most_com = find_right_most_coms(rotated_com)
        robot_com = find_robot(rotated_com)
        
        ratio = calculate_exact_ratio(rotated_data, rotated_com, second_highest_com, robot_com, right_most_com, second_right_most_com, labels)
        print(f"Group {i}: Calculated Ratio: {ratio}")
        
        # Dynamically categorize the ratio
        result = categorize_dynamically(i, ratio, categories)

        # Check if the user typed 'exit'
        if result == 'exit':
            print("Stopping further analysis.")
            break  # Exit the loop if 'exit' is encountered

        categories = result

    # Save the categories to a file after processing
    with open(save_file, 'wb') as f:
        pickle.dump(categories, f)
        print(f"Categories saved to {save_file}")

    # Generate categorized plots for the groups processed so far
    if categories:
        generate_categorized_plots(categories)
    
    # Generate Gaussian curves for each category
    if categories:
        gaussian_curves = gaussian_curve_generator(categories)
    
        # Show the Gaussian curve for the analyzed categories
        for category, curve_data in gaussian_curves.items():
            plt.plot(curve_data["x_values"], curve_data["y_values"], label=f'{category} (mean={curve_data["mean"]:.3f}, std={curve_data["std"]:.3f})')
        
        plt.title("Gaussian Curves for Categories")
        plt.xlabel("Ratio")
        plt.ylabel("Probability Density")
        plt.legend()
    
    return ratios, categories, gaussian_curves if 'gaussian_curves' in locals() else None
